In [11]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('chicago-crime-property.db')  # Replace with your actual database name

# Query all rows from the 'property_with_crime' table
query = 'SELECT * FROM property_with_crime'
property_with_crime_df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the property_with_crime DataFrame
print(property_with_crime_df)
property_with_crime_df.describe()


                style  beds  sqft  year_built  days_on_mls  sold_price  \
0       SINGLE_FAMILY     3  1350        1890          292     10155.0   
1       SINGLE_FAMILY     3  1300        1916           -1     57000.0   
2       SINGLE_FAMILY     3   692        1951          202     37000.0   
3        MULTI_FAMILY     7    -1          -1            2    175000.0   
4       SINGLE_FAMILY     2  1437        1922          122     21000.0   
...               ...   ...   ...         ...          ...         ...   
313414      TOWNHOMES     3   731        1962           -1     57000.0   
313415      TOWNHOMES     3   731        1967           67     55000.0   
313416  SINGLE_FAMILY     3  1318        1967           57    110000.0   
313417  SINGLE_FAMILY     4    -1        1969           52     90000.0   
313418  SINGLE_FAMILY     3  1152        1890           32     20000.0   

             last_sold_date   latitude  longitude       community  \
0       2014-02-11 00:00:00  41.737989 -87

,beds,sqft,year_built,days_on_mls,sold_price,latitude,longitude,crime_count
count,313419.000000,3.134190e+05,313419.000000,313419.000000,3.134190e+05,313419.000000,313419.000000,313419.000000
mean,2.156532,1.125788e+03,1304.297113,33.810659,7.236835e+05,41.881463,-87.674483,5147.207342
std,3.416618,6.579447e+03,916.872425,67.991046,5.971562e+06,0.084896,0.057103,3302.427075
min,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000e+00,41.644737,-87.935380,262.000000
25%,-1.000000,-1.000000e+00,-1.000000,-1.000000,1.890000e+05,41.824196,-87.706507,2368.000000
50%,2.000000,9.150000e+02,1915.000000,-1.000000,3.075000e+05,41.898391,-87.663889,4347.000000
75%,4.000000,1.600000e+03,1958.000000,53.000000,4.900000e+05,41.944243,-87.635302,7940.000000
max,999.000000,1.923400e+06,2023.000000,2144.000000,3.673000e+08,42.022841,-87.524883,12420.000000


In [1]:
import sqlite3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd


conn = sqlite3.connect('chicago-crime-property.db')

# Read data from the database into a DataFrame 
query = 'SELECT * FROM property_with_crime'
merged_table = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Assuming X and y are your feature matrix and target variable
X = merged_table[['crime_count', 'neighborhood']]
y = merged_table['sold_price']

# Convert 'neighborhood' column to one-hot encoding
X = pd.get_dummies(X, columns=['neighborhood'], drop_first=True)

# Specify a minimum number of samples for the training set
min_samples_train = 100 

# Set a fixed train_size
train_size = 0.8  # 80% for training

# Perform train-test split with stratification to ensure a minimum number of samples in each class
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_size, shuffle=True, random_state=42, stratify=y)

# Check if the training set has enough samples
if X_train.shape[0] < min_samples_train:
    print(f"Error: Training set has less than {min_samples_train} samples. Adjust parameters or dataset.")
else:
    # Initialize the Linear Regression model
    model = LinearRegression()

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Predict on the test data
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')

    # Optionally, you can also examine the coefficients and intercept
    coefficients = model.coef_
    intercept = model.intercept_

    # Print coefficients and intercept
    print('Coefficients:', coefficients)
    print('Intercept:', intercept)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.